In [1]:
%reload_ext autoreload
%autoreload 1

%aimport dataset

import numpy as np
import pandas as pd
import requests
from readability import Document
from sklearn.externals import joblib
from IPython.display import display

In [2]:
clf = joblib.load('models/bag-of-words-all-classes.pkl')
temas = pd.read_csv('models/temas.csv')
print('Modelo treinado para {} temas'.format(temas.shape[0]))

Modelo treinado para 39 temas


In [3]:
#os.environ['http_proxy'] = "http://127.0.0.1:3128" 
#os.environ['https_proxy'] = "https://127.0.0.1:3128"

def get_text_from_url(url):
    '''
    Get most significant text returned from URL
    '''
    response = requests.get(url)
    doc = Document(response.text)
    text = dataset.preprocess_html(doc.summary())
    return text

def inference(clf, labels, text, top_k=3):
    '''
    Returns top_k labels for text.
    '''
    vectorizer = clf.named_steps['tfidf']
    tfidf = vectorizer.transform([text])
    probs = clf.named_steps['clf'].predict_proba(tfidf)
    
    preds = np.argsort(probs, axis=1)[0,-top_k:][::-1]
    probs = probs[:, preds][0]
    pred_labels = list(map(lambda x: labels[labels.CLASS == x].TEMA.values[0], preds))
    preds_df = pd.DataFrame({'LABEL': pred_labels, 'PROBABILITY': probs})
    
    #if verbose:
    print(preds_df)
    
    terms_df = top_terms(tfidf, vectorizer.get_feature_names())
    print('\n\n')
    print(terms_df)
    
    return preds_df, terms_df

def inference_url(clf, labels, url, top_k=3, verbose=False):
    text = get_text_from_url(url)
    if verbose:
        print('Text:\n\n{}\n'.format(text))
    return inference(clf, labels, text, top_k)

def top_tfidf_feats(row, features, top_k=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_k]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['TOP TERMS', 'TF_IDF SCORE']
    return df

def top_terms(doc, features, top_k=10):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(doc.toarray())
    return top_tfidf_feats(row, features, top_k)

### Classificação de normas

In [4]:
%%time
preds = inference_url(clf, temas, 'http://www2.camara.leg.br/legin/fed/lei/1990/lei-8112-11-dezembro-1990-322161-publicacaooriginal-1-pl.html', verbose=False)

                              LABEL  PROBABILITY
0             Administração Pública     0.646957
1  Previdência e Assistência Social     0.158658
2  Direito Penal e Processual Penal     0.032886



     TOP TERMS  TF_IDF SCORE
0     servidor      0.376633
1           ou      0.271193
2        cargo      0.175954
3           ss      0.155810
4      licenca      0.142387
5         sera      0.139701
6  do servidor      0.121269
7          por      0.099847
8           se      0.095868
9           ao      0.094658
Wall time: 4.74 s


### Classificação de notícias

In [5]:
%%time
preds, terms = inference_url(clf, temas, 'http://www2.camara.leg.br/camaranoticias/noticias/538631.html', verbose=True)
#top_terms(tfidf, vectorizer.get_feature_names())

Text:

A comissao especial que analisa a proposta (PEC 15/15) que torna permanente o Fundo de Manutencao e Desenvolvimento da Educacao Basica e Valorizacao dos Profissionais da Educacao (Fundeb) promove audiencia publica nesta manha para discutir sugestoes de aprimoramento do texto e medidas de cooperacao entre os entes federados no setor educacional.

Criado em 2006 para vigorar ate 2020, o Fundeb e um fundo que utiliza recursos federais, dos estados, Distrito Federal e municipios para financiar a educacao basica no Pais, incluindo a remuneracao dos professores.
O evento e uma iniciativa da relatora da PEC, deputada Professora Dorinha Seabra Rezende (DEM-TO). Segundo ela, com o fim da vigencia do fundo se aproximando e urgente debater o assunto.
Foram convidados para a reuniao: o vice-presidente da Frente Nacional de Prefeitos (FNP), Elias Dinis; e o gestor do Observatorio de Informacoes Municipais, Francois Eugene Jean de Bremaeker.
A audiencia sera realizada no plenario 9, a partir 

In [6]:
preds = inference_url(clf, temas, 'http://g1.globo.com/sao-paulo/noticia/temer-diz-que-governo-estuda-aumento-da-aliquota-do-ir-mas-que-ainda-nao-ha-nada-decidido.ghtml', verbose=True)

Text:

  Temer diz que governo estuda aumento da aliquota do IR, mas que ainda nao ha 'nada decidido'   'Ha estudos, dos mais variados estudos', disse em evento em Sao Paulo. Medidas precisam ser aprovadas pelo Congresso.              Temer diz que governo estuda aumento da aliquota do IR, mas que ainda nao ha 'nada decidi        O presidente Michel Temer (PMDB) afirmou nesta terca-feira (8) que ha estudos sendo feitos para o aumento da aliquota do Imposto de Renda (IR), mas que nao ha nada definido. A equipe economica quer aumentar a arrecadacao - ate junho, o deficit das contas do governo era de R$ 56,092 bilhoes.     "Ha estudos, ha dos mais variados estudos. Sao estudos que se fazem rotineiramente. A todo momento estao fazendo planejamento nos setores da economia, eles fazem esses estudos. Sao estudos que estao sendo feitos, mas nada decidido", disse o presidente apos cerimonia de abertura da Fenabrave em Sao Paulo.     Parte da equipe economica decidiu tirar da gaveta projetos de 

In [24]:
%%time
preds = inference_url(clf, temas, 'http://www2.camara.leg.br/camaranoticias/noticias/DIREITOS-HUMANOS/538744-COMISSAO-REDUZ-IDADE-PARA-PESSOA-COM-DEFICIENCIA-SER-DECLARADA-IDOSA.html', verbose=False)

                                    Label  Probability
0  Direitos Humanos, Minorias e Cidadania     0.440917
1                   Administração Pública     0.043851
2                    Processo Legislativo     0.042497



         top terms  tf-idf score
0  com deficiencia      0.339010
1      deficiencia      0.327722
2      pessoas com      0.182542
3   envelhecimento      0.171066
4       pessoa com      0.167184
5            idosa      0.152612
6         deputada      0.144467
7       tipicas do      0.132910
8          50 anos      0.132910
9         relatora      0.132910
Wall time: 4.45 s


In [25]:
%%time
preds = inference_url(clf, temas, 'http://www2.camara.leg.br/camaranoticias/noticias/TRABALHO-E-PREVIDENCIA/538706-COMISSAO-DE-TRABALHO-APROVA-A-REGULAMENTACAO-DA-PROFISSAO-DE-GARCOM-COM-PISO-SALARIAL-DE-R$-2.811.html', verbose=False)

                     Label  Probability
0       Trabalho e Emprego     0.601329
1    Administração Pública     0.037124
2  Relações Internacionais     0.032458



               top terms  tf-idf score
0           duas emendas      0.237887
1                   piso      0.183349
2          piso salarial      0.151964
3  regulamenta profissao      0.130203
4                afirmou      0.125221
5        acrescimo sobre      0.125221
6                emendas      0.119785
7                 de 811      0.116703
8                  texto      0.115865
9         texto aprovado      0.108186
Wall time: 4.49 s
